# IIC2413 - Control 2: SQL e Índices

## Entrega
El plazo para la entrega es Lunes, 3 de Mayo, hasta las 20:00. El control deben hacerlo rellenando este mismo archivo `.ipynb` y  luego subirlo al siding (se abrirá un cuestionario para esta entrega).

## Parte 1: Consultas SQL (5 puntos) 

El objetivo de esta sección es que tengan un primer acercamiento a lo que es consultar una base de datos real de una aplicación en producción.
Para eso usaremos una versión muy reducida y anonimizada de la base de datos de la start up [TuShop](https://www.tushop.cl/) cuyo sofware genera automágicamente una página web con carrito de compras diseñada especialmente para tiendas de instagram. La base está lista para ser accedida con sqlite abriendo el archivo `tu_shop_mini.db` disponible en el repositorio del curso junto con este enunciado.

El esquema de la base de datos es el siguiente:
```
Stores(id, email, name, created_at)
Products(id, name, store_id, row_order, created_at)
Bills(id, email, address, store_id, status, created_at)
Variants(id, product_id, is_master, price, quantity, name_suffix, created_at)
BillVariants(id, bill_id, quantity, price, variant_id, discount_rate, created_at)
```
Mas detalles de lo que representa cada tabla y columna en el archivo `metadata.md` también disponible en el repositorio junto con este enunciado.

### Setup

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///tu_shop_mini.db

In [ ]:
%sql SELECT COUNT(*) FROM Bills

Si la respuesta a la consulta de arriba es 2767 entonces estás listo para responder el control.

### Consultas

**1.** El id y nombre de cada producto junto con el precio y cantidad de su master variant  ordenados ascendente por 'row_order'. **(0.3 pts)**

In [ ]:
%%sql

**2.** El status y la fecha de creacion de las ventas de la tienda `tío zetamart` en marzo del 2021, ordenados por la fecha de creacion. **(0.3 pts)**

In [ ]:
%%sql

**3.** Los emails de las personas que hayan comprado al menos una vez en la tienda `pc_faketory` antes de abril del 2021. **(0.3 pts)**

In [ ]:
%%sql

**4.** Los nombres completos, precio y cantidad y valor total (precio * cantidad) de cada variante. **(0.3 pts)**

In [ ]:
%%sql

**5.** El nombre de la tienda y la dirección de envío de las ventas que ya hayan sido enviadas (`shipped`) y tengan 'Cádiz' en la dirección. **(0.3 pts)**

In [ ]:
%%sql

**6.** Los nombres de productos que sean vendidos por la tienda `tío zetamart` y la tienda `pc_faketory`. Ten en cuenta que cuando decimos "productos que sean vendidos por ambas tiendas", son en realidad los productos que tengan el mismo nombre. **(0.4 pts)**

In [ ]:
%%sql

**7.** La cantidad de unidades vendidas de cada producto en cada tienda. **(0.4 pts)**


In [ ]:
%%sql

**8.** El valor de las ventas de cada tienda en marzo (en $$$), ordenadas por ventas. **(0.4 pts)** **Pro Tip:** Para dividir dos ints y que el resultado sea float hay que convertir uno de los ints a float primero, lo que se puede hacer multiplicando por `1.0`. Algo así: `columna_a*1.0/100`

In [ ]:
%%sql

**9.** Los ids de las variantes que no han tenido ninguna venta en marzo y tienen stock disponible. **(0.4 pts)**

In [ ]:
%%sql

**10.** El nombre y precio de la variante con el precio más caro. **(0.4 pts)**

In [ ]:
%%sql

De las siguientes 4 consultas solo debes hacer 3. Si haces las 4 no te revisaremos la última.

**11.** El valor promedio de venta de cada tienda en marzo (con sus respectivos nombres o ids). **(0.5 pts)**

In [ ]:
%%sql

**12.** El porcentaje del total de clientes que fueron recurrentes en el mes de abril (recurrentes son aquellos que ya habían comprado alguna vez antes de comprar en abril). Los clientes se identifican por el email **(0.5 pts)**

In [ ]:
%%sql

**13.**  La cantidad de dias en promedio que han pasado desde que se creo cada venta, agrupado por status. **(0.5 pts)**

**Pro Tips**:
- `DATE(SUBSTR(columna, 0, 11))` Retorna un Date de sqlite a partir de una columna de fecha de nuesta base de datos.
-`DATE('now')` Retorna un objeto Date de sqlite con la fecha de hoy
-La función [JULIANDAY(date)](https://www.techonthenet.com/sqlite/functions/julianday.php#:~:text=The%20SQLite%20julianday%20function%20takes,as%20a%20floating%20point%20number.) Retorna los días que han pasado entre date y una fecha constante muy en el pasado.

In [ ]:
%%sql

**14.** El nombre de la tienda que más ventas hizo en abril, junto con la cantidad de ventas. **Pro Tip**: La sentencia [LIMIT](https://www.sqlitetutorial.net/sqlite-limit/) que reduce la cantidad de resultados podría servir.

In [ ]:
%%sql

## Parte 2: Índices (1 punto, 0.1 cada respuesta) 

En esta parte del control tienen que responder cuál será el costo I/O para ejecutar consultas SQL ocupando distintos tipos de índices.

En particular, en este ejercicio asumiremos que tenemos una relación ``Alumno(id INT PRIMARY KEY, nombre VARCHAR(100), promedio FLOAT)``.

Tenemos a nuestra disposición dos índices: 
- Un B+tree *clustered* sobre id, y 
- Un hash index *unclustered* sobre el promedio.

La relación tiene 320,000 tuplas. En cada página del disco nos alcanzan 100 tuplas.El promedio toma valores entre 3.9 y 7.0, en incrementos de 0.1 (i.e. hay solo 32 posibles valores de promedio). Los ids van desde 1 a 320,000.

El árbol es de altura *h*, y la hojas están ocupadas al 80%. 

En el Hash Index cada página contiene 1,000 punteros a las páginas del disco y hay 32 buckets (uno para cada promedio). Las notas son distribuidas de manera uniforme y **sí hay overflow pages** (deberían calcular cuánto).

¿Cuál es el costo de evaluar las siguientes consultas con cada índice (en el peor de los casos)?

```sql
1. SELECT * FROM Alumno WHERE id=12
2. SELECT * FROM Alumno WHERE nombre = 'Adrián Soto'
3. SELECT * FROM Alumno WHERE promedio = 5.6
4. SELECT * FROM Alumno WHERE id > 1000 AND id <= 2000
5. SELECT * FROM Alumno WHERE promedio <4.9
```